In [1]:
# this sets up basic packages
import numpy as np
import pandas as pd
import astropy.units as u
import astropy.cosmology.units as cu

# this sets up matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# this sets up astropy
from astropy.io import fits
from astropy.table import Table
from astropy.wcs import WCS
from astropy.wcs.utils import pixel_to_skycoord, skycoord_to_pixel
from astropy.utils.data import get_pkg_data_filename
from astropy.coordinates import SkyCoord, Angle, match_coordinates_sky, Distance

In [2]:
# this reads in the new neighbor catalog, originally made for CIGALE
df = pd.read_csv('post_correction_data.csv')

In [3]:
print('Before all the eliminations, there are a total of', df.shape[0], 'sources.')

Before all the eliminations, there are a total of 994 sources.


# <b>CRITERION 1</b>: Minimum of 3 detections, or less if there's A3COSMOS/ALMA data

In [5]:
# in a previous .csv file, all the neighbors are id'd based on their central galaxy.
# since this id is such a minor part of our work and isn't present in our later .csv or .txt files,
# we just retrieve these "our_id" values from the old .csv file, and then use them for our new work.
df2 = pd.read_csv('neighbor_data.csv')
catalog_id = df2[df2['id'].duplicated(keep='first')==False]['id']
our_id = df2[df2['id'].duplicated(keep='first')==False]['our_id']
our_ra = df2[df2['id'].duplicated(keep='first')==False]['ra']
our_dec = df2[df2['id'].duplicated(keep='first')==False]['dec']
our_z = df2[df2['id'].duplicated(keep='first')==False]['z_spec']

In [6]:
# this opens the HDU list of the .fits catalog from A3COSMOS
hdu_list = fits.open('/Volumes/LaCie/COSMOS_DATA/a3cosmos_blind.fits')
hdu_list.info()

# this gets the data of the catalog
a3cosmos = hdu_list[1].data

Filename: /Volumes/LaCie/COSMOS_DATA/a3cosmos_blind.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      45   ()      
  1  J_ApJS_244_40_blind    1 BinTableHDU    115   1134R x 16C   [D, D, D, D, E, E, E, E, E, E, E, D, E, E, E, B]   


In [7]:
### matching coords from the A3COSMOS catalog using SkyCoord 
# create SkyCoord arrays (?) with the RA and Dec of the galaxies in both catalogs
radio_cat = SkyCoord(ra=a3cosmos['RAJ2000']*u.degree, dec=a3cosmos['DEJ2000']*u.degree)
spec_cat = SkyCoord(ra=our_ra.values*u.degree, dec=our_dec.values*u.degree)

# use search_around_sky to find matching indices (matching in RA and Dec) in each catalog
idx_radio, idx_spec, d2d, d3d = spec_cat.search_around_sky(radio_cat, 1*u.arcsec)

# use the matching indices to the new catalog to see where we have ground-based data of our neighbors
print(np.size(idx_radio), 'is the number of matches between the A3COSMOS and the spectroscopic catalogs.')

2 is the number of matches between the A3COSMOS and the spectroscopic catalogs.


In [8]:
# this calls the names of all the columns with flux measurements for all the galaxies
f_cols = [col for col in df.columns if '_err' not in col and col != 'id' and col != 'redshift']

In [9]:
# this saves all the neighbor ids that pass criterion 1
crit1_neigh_id = []

# this loops over all the galaxies from the .csv file, then reads their corresponding fluxes
# in all our bands to see if each of them has at least 3 different detections. if said galaxy does, 
# then we save the id of that galaxy, meaning that it passes criterion 1.

# this loops over each of the 9 QGs in our sample group
for id in catalog_id.values:
    
    # this retrieves all the corresponding fluxes of the galaxy with the id called
    all_bands = df[df['id'] == id][f_cols].values

    # this counts how many of these fluxes are non-zeros (i.e. how many detections there are)
    available_fluxes = np.count_nonzero(all_bands)

    # this adds the galaxy's id into crit1_neigh_id if the galaxy has more than 3 detections
    if available_fluxes >= 3:
        crit1_neigh_id.append(id)
        
print(len(crit1_neigh_id), 'is the total amount of neighbors that pass criterion 1.')

849 is the total amount of neighbors that pass criterion 1.


# <b>CRITERION 2:</b> |z$_{QG}$ - z$_{gal}$| < 0.5

In [11]:
# this reads in the results FITS file that CIGALE made
cigale_results = fits.open('out/results.fits')
cigale_results.info()

# this saves the WCS setting to be used later, just in case
cigale_wcs = WCS(cigale_results[1].header)

Filename: out/results.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU    386   994R x 127C   [K, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D]   


In [32]:
# this gets the data from the results FITS file
cigale_data = Table(cigale_results[1].data).to_pandas()

In [34]:
# this selects only the rows in cigale_data and our_id
# whose corresponding source already passed criterion 1
reduced_cigale = cigale_data.loc[cigale_data['id'].isin(crit1_neigh_id)]
reduced_our_id = our_id[catalog_id.isin(crit1_neigh_id)]

In [36]:
cigale_id = reduced_cigale['id'].values
cigale_z = reduced_cigale['bayes.universe.redshift'].values
cigale_z_err = reduced_cigale['bayes.universe.redshift_err'].values

In [38]:
# now read in the RAs and Decs of our 9 galaxies
info = np.genfromtxt('basic_data.txt', delimiter=' ', dtype=['U15', '<f8','<f8', '<f8'])

# this gets the galaxy redshifts for all the 9 galaxies
galaxy_redshift = np.zeros(np.size(info), dtype=float)
for i in range(np.size(info)):
    galaxy_redshift[i] = info[i][3]

In [42]:
# this saves all the neighbor ids that pass criterion 2
crit2_neigh_id = []

# this loops over each of the 9 QGs in our sample group
for our_galaxy in range(np.size(galaxy_redshift)):
    
    # this selects the neighbor IDs, ONLY the ones that passed criterion 1
    neigh_id = np.array(cigale_id[reduced_our_id==our_galaxy])

    # this reads in the estimated z and errors of all the neighbors around each galaxy that passed criterion 1
    neigh_z = np.array(cigale_z[reduced_our_id==our_galaxy])
    neigh_z_err = np.array(cigale_z_err[reduced_our_id==our_galaxy])

    # this sets criterion 2
    criteria = (np.abs(neigh_z + neigh_z_err - galaxy_redshift[our_galaxy]) <= 0.5) | (np.abs(neigh_z - neigh_z_err - galaxy_redshift[our_galaxy]) <= 0.5)

    # this finds neighbor IDs that pass criterion 2
    ok_neigh_id = neigh_id[criteria]

    # this saves the indices of the neighbors that pass criterion 2
    crit2_neigh_id.extend(ok_neigh_id)

print(len(crit2_neigh_id), 'is the total amount of neighbors that pass criterion 2.')

160 is the total amount of neighbors that pass criterion 2.
